# Neural Networks: Simple

## Chapter 7

### Dot Product

In [1]:
from typing import List
from math import exp

def dot_product(xs: List[Float], ys: List[Float]) -> float:
    """
    Dot product of two lists.
    """
    return sum(x * y for x, y in zip(xs, ys))

### Sigmoid Activation Function

In [2]:
def sigmoid(x: float) -> float:
    """
    Sigmoid activation function.
    """
    return 1 / (1 + exp(-x))

def derivative_sigmoid(x: float) -> float:
    """
    Derivative of sigmoid activation function.
    """
    return sigmoid(x) * (1 - sigmoid(x))

## Neuron

Must have 

* weights
* delta
* learning_rate
* cache of last output
* activation_function
* derivative of activation_function

In [3]:
from typing import Callable


class Neuron:
    """
    Basic unit for a neural network. Must have weights, activation function, derivative of activation function,
    learning rate, cache of last output, and delta.
    """

    def __init__(
        self,
        weights: List[float],
        learning_rate: float,
        activation_function: Callable[[float], float],
        derivative_activation_function: Callable[[float], float],
    ) -> None:
        self.weights: List[float] = weights
        self.activation_function: Callable[[float], float] = activation_function
        self.derivative_activation_function: Callable[
            [float], float
        ] = derivate_activation_function
        self.learning_rate: float = learning_rate
        self.output_cache: float = 0.0
        self.delta: float = 0.0

    def output(self, inputs: List[float]) -> float:
        """
        Feed forward pass of neuron.
        """
        self.output_cache = dot_product(inputs, self.weights)
        return self.activation_function(self.output_cache)

## Layer

A layer must have:

* neurons
* output cache (after the activation function is applied to neuron's output)
* previous layer

In [4]:
from __future__ import annotations
from typing import Optional
from random import random


class Layer:
    """
    Base class for a neural network layer. Must know the previous layer, the neurons, and an output cache (after activation function).
    """

    def __init__(
        self,
        previous_layer: Optional[Layer],
        num_neurons: int,
        learning_rate: float,
        activation_function: Callable[[float], float],
        derivative_activation_function: Callable[[float], float],
    ) -> None:
        self.previous_layer: Optional[Layer] = previous_layer
        self.neurons: List[Neuron] = []

        # Add neurons
        for i in range(num_neurons):
            if previous_layer is None:
                random_weights: List[float] = []
            else:
                # Each neuron is connected to every neuron in previous layer
                random_weights = [random() for _ in range(len(previous_layer.neurons))]
            neuron: Neuron = Neuron(
                random_weights,
                learning_rate,
                activation_function,
                derivative_activation_function,
            )
            self.neurons.append(neuron)
        # Initialize empty output cache
        self.output_cache: List[float] = [0.0 for _ in range(num_neurons)]

    def outputs(self, inputs: List[float]) -> List[float]:
        """
        Calculate outputs of all neurons in layer.
        """
        if self.previous_layer is None:  # This is an input layer
            self.output_cache = inputs
        else:
            self.output_cache = [n.output(inputs) for n in self.neurons]
        return self.output_cache

    def calculate_deltas_for_output_layer(self, expected: List[float]) -> None:
        """
        Calculate deltas for output layer. Delta = f'(output_cache) * error
        
        f'(output_cache) is derivative of activation with respect to output cache
        error is expected - actual
        """
        for n in range(len(self.neurons)):
            # Call the derivative of the activation function on the neuron's output and
            # multiply by the expected value
            self.neurons[n].delta = self.neurons[n].derivative_activation_function(
                self.neurons[n].output_cache
            ) * (expected[n] - self.output_cache[n])

    def calculate_deltas_for_hidden_layer(self, next_layer: Layer) -> None:
        """
        Calculate deltas for hiddne layer. Delta = f'(output_cache) * (next_weights * next_deltas)
        """
        for index, neuron in enumerate(self.neurons):
            next_weights: List[float] = [n.weights[index] for n in next_layer.neurons]
            next_deltas: List[float] = [n.delta for n in next_layer.neurons]
            sum_weights_and_deltas: float = dot_product(next_weights, next_deltas)
            neuron.delta = (
                self.derivative_activation_function(neuron.output_cache)
                * sum_weights_and_deltas
            )

## Network

In [7]:
from typing import TypeVar, Tuple
from functools import reduce

# Output type of interpretation of neural network
T = TypeVar("T")


class Network:
    """
    Base class for neural network. Must keep state of layers.
    """

    def __init__(
        self,
        layer_structure: List[int],
        learning_rate: float,
        activation_function: Callable[[float], float] = sigmoid,
        derivative_activation_function: Callable[[float], float] = derivative_sigmoid,
    ) -> None:
        if len(layer_structure) < 3:
            raise ValueError(
                "Error: Should be at least 3 layers (input, hidden, output)"
            )

        self.layers: List[layer] = []
        # Input layer
        input_layer: Layer = Layer(
            previous_layer=None,
            num_neurons=layer_structure[0],
            learning_rate=learning_rate,
            activation_function=activation_function,
            derivative_activation_function=derivative_activation_function,
        )
        self.layers.append(input_layer)

        # Hidden layers and output layer
        for previous, num_neurons in enumerate(layer_structure[1::]):
            next_layer = Layer(
                previous_layer=self.layers[previous],
                num_neurons=num_neurons,
                learning_rate=learning_rate,
                activation_function=activation_function,
                derivative_activation_function=derivative_activation_function,
            )
            self.layers.append(next_layer)

    def output(self, input: List[float]) -> List[float]:
        """
        Calculate outputs for entire network.
        """
        # Pushes input data to first layer, then output from first as input to second, output from second
        # as input to third and so on
        return reduce(
            function=lambda inputs, layer: layer.outputs(inputs),
            sequence=self.layers,
            initial=input,
        )

    def backpropagate(self, expected: List[float]) -> None:
        """
        Calculate delta (change) for each neuron in each layer. Move backwards through the network
        from output towards input.
        """
        # Calculate delta for output
        last_layer: int = len(self.layers) - 1
        self.layers[last_layer].calculate_deltas_for_output_layer(expected)

        # Calculate delta for hidden layers moving from end to beginning
        for l in range(last_layer - 1, 0, -1):
            # Send in previous layer
            self.layers[l].calculate_deltas_for_hidden_layer(self.layers[l + 1])

    def update_weights(self) -> None:
        """
        Apply formula to update the weights of all neurons.
        
        weight = weight + learning_rate * input * neuron_delta
        """
        for layer in self.layers[1:]:
            for neuron in layer.neurons:
                for w in range(len(neuron.weights)):
                    neuron.weights[w] = neuron.weights[w] + (
                        neuron.learning_rate
                        * layer.previous_layer.output_cache[w]
                        * neuron.delta
                    )
                    
    def train(self, inputs: List[List[float]], expecteds: List[List[float]]) -> None:
        """
        Train the network to map from the inputs to the expecteds using the neuron's weights.
        """
        for location, xs in enumerate(inputs):
            ys: List[float] = expecteds[location]
            outs: List[float] = self.outputs(xs)
            # Backpropagate to calculate deltas
            self.backpropagate(ys)
            # Update weights with deltas
            self.update_weights()
            
    def validate(self, inputs: List[List[float]], expecteds: List[T], interpret_output: Callable[[List[float]], T]) -> Tuple[int, int, float]:
        """
        Validate the network's outputs on a dataset. Returns the correct number of trials and the percentage
        correct out of the total. Only applicate for classification tasks. 
        The callable must interpret the outputs in the problem context.
        """
        correct: int = 0
        for input, expected in zip(inputs, expecteds):
            # Calculate output from input
            outs: List[float] = self.outputs(input)
            # Interpret the floats in the data context.
            result: T = interpret_output(outs)
                
            if result == expected:
                correct += 1
            
        percentage: float = correct / len(inputs)
        return correct, len(inputs), percentage

## Feature Scaling

We will scale inputs to between 0 and 1 for input to the neural network.

In [15]:
def normalize_by_feature_scaling(dataset: List[[List[float]]]) -> None:
    for col_num in range(len(dataset[0])):
        column: List[float] = [row[col_num] for row in dataset]
        maximum, minimum = max(column), min(column)
        for row_num in range(len(column)):
            dataset[row_num][col_num] = (dataset[row_num][col_num] - minimum) / (maximum - minimum)
            
d = [[2, 4, 3, 8], [4, 6, 2, 7], [3, 5, 2.5, 7.5]]
normalize_by_feature_scaling(d)
d

[[0.0, 0.0, 1.0, 1.0], [1.0, 1.0, 0.0, 0.0], [0.5, 0.5, 0.5, 0.5]]

In [18]:
reduce(lambda input, output: input + output, [4, 5, 6], 3)

18

## Testing with Classic Iris Dataset

This dataset serves as a benchmark for classification models. It is simple, yet will let us ascertain if our model is behaving as expected.

In [22]:
import csv
from random import shuffle

def read_iris():
    """
    Read in the iris data file as list of lists.
    """
    iris_parameters: List[List[float]] = []
    iris_classifications: List[List[float]] = []
    iris_species: List[str] = []
        
    with open('iris.csv', mode='r') as iris_file:
        irises: List = list(csv.reader(iris_file))
        shuffle(irises) # Mix randomly
        for iris in irises:
            # The first 4 columns are the features
            parameters: List[float] = [float(n) for n in iris[:4]]
            iris_parameters.append(parameters)
            species: str= iris[4]
            if species == 'Iris-setosa':
                iris_classifications.append([1.0, 0.0, 0.0])
            elif species == 'Iris-versicolor':
                iris_classifications.append([0.0, 1.0, 0.0])
            elif species == 'Iris-virginica':
                iris_classifications.append([0.0, 0.0, 1.0])
            iris_species.append(species)
            
    normalize_by_feature_scaling(iris_parameters)
    
    return iris_parameters, iris_classifications, iris_species

iris_parameters, iris_classifications, iris_species = read_iris()

In [23]:
from collections import Counter
Counter(iris_species)

Counter({'Iris-versicolor': 50, 'Iris-setosa': 50, 'Iris-virginica': 50})

In [26]:
max([n[0] for n in iris_parameters])
min([n[0] for n in iris_parameters])

max([n[1] for n in iris_parameters])
min([n[1] for n in iris_parameters])

1.0

0.0

1.0

0.0

### Implement Interpretation of Model Output for Dataset